# Zadanie 1. (4+Xp)

Zajmiemy się osadzeniami słów (zarówno kontekstowymi, jak i bezkontekstowymi). Uwaga: teksty, które będziemy osadzać zawsze składają się z jednego słowa (ale niekoniecznie z jednego tokenu).

**a)** Zaproponuj jakiś sposób wykorzystania bezkontekstowych osadzeń tokenów (wyznaczanych przez transformer) do wyznaczenia osadzeń słów. Możesz skorzystać z programu z wykładu 7 (`embedding.ipynb`). Sprawdź, jaką jakość (mierzoną testem ABX) mają te osadzenia. Do zaliczenia zadania wymagane jest 0.6.

**b)** Wykorzystaj kontekstowe osadzenia tokenów z BERT-a do wyznaczenia osadzeń dla słów. Ponownie wykonaj testy ABX.

**c)** Spróbuj połączyć te dwa podejścia w jakikolwiek sposób. Jakość twojego rozwiązania przekłada się na punkty bonusowe zgodnie z wzorem: `(score − 0.6) × 6`

**Procedura ewaluacji** (być może zostanie uproszczona): osadzenia zapisz w pliku tekstowym `word_embedings_file.txt`, w którym każdy wiersz wygląda tak:

```
[słowo] float_1 float_2 ... float_D
```

Osadzenia są oceniane za pomocą skryptu `word_emb_evaluation.py`.

In [220]:
clusters_txt = '''
piśmiennicze: pisak flamaster ołówek długopis pióro
małe_ssaki: mysz szczur chomik łasica kuna bóbr
okręty: niszczyciel lotniskowiec trałowiec krążownik pancernik fregata korweta
lekarze: lekarz pediatra ginekolog kardiolog internista geriatra
zupy: rosół żurek barszcz
uczucia: miłość przyjaźń nienawiść gniew smutek radość strach
działy_matematyki: algebra analiza topologia logika geometria
budynki_sakralne: kościół bazylika kaplica katedra świątynia synagoga zbór
stopień_wojskowy: chorąży podporucznik porucznik kapitan major pułkownik generał podpułkownik
grzyby_jadalne: pieczarka borowik gąska kurka boczniak kania
prądy_filozoficzne: empiryzm stoicyzm racjonalizm egzystencjalizm marksizm romantyzm
religie: chrześcijaństwo buddyzm islam prawosławie protestantyzm kalwinizm luteranizm judaizm
dzieła_muzyczne: sonata synfonia koncert preludium fuga suita
cyfry: jedynka dwójka trójka czwórka piątka szóstka siódemka ósemka dziewiątka
owady: ważka biedronka żuk mrówka mucha osa pszczoła chrząszcz
broń_biała: miecz topór sztylet nóż siekiera
broń_palna: karabin pistolet rewolwer fuzja strzelba
komputery: komputer laptop kalkulator notebook
kolory: biel żółć czerwień błękit zieleń brąz czerń
duchowny: wikary biskup ksiądz proboszcz rabin pop arcybiskup kardynał pastor
ryby: karp śledź łosoś dorsz okoń sandacz szczupak płotka
napoje_mleczne: jogurt kefir maślanka
czynności_sportowe: bieganie skakanie pływanie maszerowanie marsz trucht
ubranie:  garnitur smoking frak żakiet marynarka koszula bluzka sweter sweterek sukienka kamizelka spódnica spodnie
mebel: krzesło fotel kanapa łóżko wersalka sofa stół stolik ława
przestępca: morderca zabójca gwałciciel złodziej bandyta kieszonkowiec łajdak łobuz
mięso_wędliny wieprzowina wołowina baranina cielęcina boczek baleron kiełbasa szynka schab karkówka dziczyzna
drzewo: dąb klon wiąz jesion świerk sosna modrzew platan buk cis jawor jarzębina akacja
źródło_światła: lampa latarka lampka żyrandol żarówka reflektor latarnia lampka
organ: wątroba płuco serce trzustka żołądek nerka macica jajowód nasieniowód prostata śledziona
oddziały: kompania pluton batalion brygada armia dywizja pułk
napój_alkoholowy: piwo wino wódka dżin nalewka bimber wiśniówka cydr koniak wiśniówka
kot_drapieżny: puma pantera lampart tygrys lew ryś żbik gepard jaguar
metal: żelazo złoto srebro miedź nikiel cyna cynk potas platyna chrom glin aluminium
samolot: samolot odrzutowiec awionetka bombowiec myśliwiec samolocik helikopter śmigłowiec
owoc: jabłko gruszka śliwka brzoskwinia cytryna pomarańcza grejpfrut porzeczka nektaryna
pościel: poduszka prześcieradło kołdra kołderka poduszeczka pierzyna koc kocyk pled
agd: lodówka kuchenka pralka zmywarka mikser sokowirówka piec piecyk piekarnik
'''

task a

In [227]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
import numpy as np
from tqdm.auto import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device}")

gpt2_model_name = 'gpt2'
gpt2_tokenizer = AutoTokenizer.from_pretrained(gpt2_model_name)
gpt2_model = AutoModelForCausalLM.from_pretrained(gpt2_model_name).to(device)

# non-contextual token embeddings from GPT-2
gpt2_embeddings = gpt2_model.transformer.wte.weight.detach().cpu().numpy()
print(f"Embeddings shape: {gpt2_embeddings.shape}")

# unique words
words = set()
for line in test_words.split('\n'):
    parts = line.split()
    if len(parts) < 2:
        continue
    words.update(parts[1:])

print(f"Unique words: {len(words)}")


Using cpu
Embeddings shape: (50257, 768)
Unique words: 288


In [326]:
def get_non_contextual_word_embedding(word, tokenizer, embeddings, method='mean'):
    """
    Get non-contextual word embedding by aggregating token embeddings
    or by isolated transformer hidden states (method='combine').
    """

    # ---------- TOKEN-LEVEL METHODS ----------
    tokens = tokenizer.tokenize(' ' + word)
    if not tokens:
        tokens = tokenizer.tokenize(word)

    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    if not token_ids:
        return None

    token_embeddings = embeddings[token_ids]  # [T, D]

    if method == 'mean':
        return token_embeddings.mean(axis=0)

    elif method == 'first':
        return token_embeddings[0]

    elif method == 'weighted':
        size = len(token_embeddings)
        N = min(4, size)

        weights = np.array([0.1, 0.5, 0.8, 1.][-N:])
        weights /= weights.sum()

        vec = np.zeros(token_embeddings.shape[1])
        for i in range(N):
            vec += token_embeddings[size - i - 1] * weights[-i-1]

        return vec

    elif method == 'positional':
        D = token_embeddings.shape[1]
        pos = np.arange(len(token_embeddings))
        pos_enc = np.sin(pos[:, None] / (10000 ** (2 * np.arange(D) / D)))
        return (token_embeddings * (1 + 0.1 * pos_enc)).mean(axis=0)

    # ---------- TRANSFORMER-LEVEL METHOD ----------
    elif method == 'combine':
        gpt2_model.eval()

        inputs = tokenizer(
            ' ' + word,
            return_tensors='pt',
            add_special_tokens=False
        ).to(device)

        with torch.no_grad():
            outputs = gpt2_model.transformer(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                output_hidden_states=True
            )

        # hidden_states: tuple[L] of [1, T, D]
        hs = torch.stack(outputs.hidden_states).squeeze(1)  # [L, T, D]

        # mean of last 4 layers
        last4 = hs[-4:].mean(dim=0)                          # [T, D]

        vec = last4[-1].cpu().numpy()
        return vec / (np.linalg.norm(vec) + 1e-9)

    else:
        raise ValueError(f"Unsupported method: {method}")


In [338]:
embeddings_1a = {}
for word in tqdm(words):
    emb = get_non_contextual_word_embedding(word, gpt2_tokenizer, gpt2_embeddings, method='combine')
    if emb is not None:
        embeddings_1a[word] = emb

print(f"Generated embeddings for {len(embeddings_1a)}/{len(words)} words")


  0%|          | 0/288 [00:00<?, ?it/s]

Generated embeddings for 288/288 words


In [339]:
X = np.stack(list(embeddings_1a.values()))
mu = X.mean(axis=0)

# subtract + normalize
for w in embeddings_1a:
    v = embeddings_1a[w] - mu
    embeddings_1a[w] = v / (np.linalg.norm(v) + 1e-9)


In [340]:
def save_embeddings(embeddings_dict, filename):
    with open(filename, 'w') as f:
        for word, emb in embeddings_dict.items():
            # format: word float1 float2 ... floatD
            emb_str = ' '.join(map(str, emb))
            f.write(f"{word} {emb_str}\n")
    print(f"Saved {len(embeddings_dict)} embeddings to {filename}")

In [341]:
save_embeddings(embeddings_1a, 'word_embedings_file.txt')
print("Embeddings saved")


Saved 288 embeddings to word_embedings_file.txt
Embeddings saved


In [342]:
!python3 word_emb_evaluation.py

PROBLEMS: 0.0
Start
TOTAL SCORE: 0.61432


task b

In [343]:
bert_model_name = 'distilbert-base-multilingual-cased'
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
bert_model = AutoModel.from_pretrained(bert_model_name, output_hidden_states=True).to(device)
bert_model.eval()

def get_contextual_word_embedding(word, tokenizer, model, device, layer=-1):
    """
    Get contextual word embedding from BERT.
    Places word in minimal context to get contextual representation.
    """
    text = f"To jest {word}."

    inputs = tokenizer(text, return_tensors='pt').to(device)

    # get embeddings from specified layer
    with torch.no_grad():
        outputs = model(**inputs)
        hidden_states = outputs.hidden_states
    layer_embedding = hidden_states[layer]

    # find the tokens that correspond to the word
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

    word_tokens = tokenizer.tokenize(word)

    word_token_indices = []
    for i in range(len(tokens)):
        if tokens[i] in ['[CLS]', '[PAD]', '[SEP]']:
            continue

        # check if this could be the start of our word
        match = True
        for j, wt in enumerate(word_tokens):
            if i + j >= len(tokens) or tokens[i + j] != wt:
                match = False
                break
        if match:
            word_token_indices = list(range(i, i + len(word_tokens)))
            break

    # if we couldn't find the word, just take the middle tokens (skip [CLS])
    if not word_token_indices:
        word_token_indices = list(range(1, len(tokens) - 1))

    # average embeddings of word tokens
    if word_token_indices:
        word_embedding = layer_embedding[0, word_token_indices, :].mean(dim=0).cpu().numpy()
        return word_embedding

    return None

In [344]:
embeddings_1b = {}
for word in tqdm(words, desc="Generating contextual embeddings"):
    emb = get_contextual_word_embedding(word, bert_tokenizer, bert_model, device)
    if emb is not None:
        # normalize embedding
        emb = emb / np.linalg.norm(emb)
        embeddings_1b[word] = emb

print(f"Successfully generated embeddings for {len(embeddings_1b)}/{len(words)} words")

save_embeddings(embeddings_1b, 'word_embedings_file.txt')
print("Embeddings saved!")

Generating contextual embeddings:   0%|          | 0/288 [00:00<?, ?it/s]

Successfully generated embeddings for 288/288 words
Saved 288 embeddings to word_embedings_file.txt
Embeddings saved!


In [345]:
!python3 word_emb_evaluation.py

PROBLEMS: 0.0
Start
TOTAL SCORE: 0.688636


task c

In [346]:
def combine_embeddings(emb_noncontextual, emb_contextual, method='weighted', weight_contextual=0.5):
    """
    Combine non-contextual and contextual embeddings
    Methods: 'weighted', 'concat'
    """
    if emb_noncontextual is None or emb_contextual is None:
        return None

    # normalize
    emb_nc = emb_noncontextual / np.linalg.norm(emb_noncontextual)
    emb_c = emb_contextual / np.linalg.norm(emb_contextual)

    if method == 'weighted':
        # weighted average
        combined = (1 - weight_contextual) * emb_nc + weight_contextual * emb_c
        return combined / np.linalg.norm(combined)
    elif method == 'concat':
        # concatenation
        combined = np.concatenate([emb_nc, emb_c])
        return combined / np.linalg.norm(combined)
    else:
        return emb_nc


In [347]:
embeddings_1c = {}
for word in tqdm(words, desc="Generating combined embeddings"):
    if word in embeddings_1a and word in embeddings_1b:
        combined_emb = combine_embeddings(embeddings_1a[word], embeddings_1b[word], method='weighted', weight_contextual=0.5)
        if combined_emb is not None:
            embeddings_1c[word] = combined_emb

print(f"Successfully generated combined embeddings for {len(embeddings_1c)}/{len(words)} words")


save_embeddings(embeddings_1c, 'word_embedings_file.txt')
print("Embeddings saved")


Generating combined embeddings:   0%|          | 0/288 [00:00<?, ?it/s]

Successfully generated combined embeddings for 288/288 words
Saved 288 embeddings to word_embedings_file.txt
Embeddings saved


In [348]:
!python3 word_emb_evaluation.py

PROBLEMS: 0.0
Start
TOTAL SCORE: 0.662298


# Zadanie 2. (6+Xp)

W zadaniu tym będziemy zajmować się klasyfikacją recenzji z wykorzystaniem modeli transformer, możesz tu skorzystać z programu z wykładu (`herbert.ipynb`). W tym zadaniu powinieneś użyć trzech modeli:

1. Modelu generatywnego, takiego jak Papuga, Polka o wielkości do 1B, który znajduje prawdopodobieństwa tekstu (podobnie, jak na liście 1)
2. Kodera typu BERT (np. herbert), jako ekstraktora cech
3. Tradycyjnego modelu Machine Learning, który integruje wyniki dwóch poprzednich modeli.

Ten model powinieneś wytrenować na zbiorze treningowym recenzji, a testować na testowym.

Wartość premii jest równa: `20 × (a − 0.85)`, gdzie `a` to wartość accuracy na zbiorze testowym.

Jeżeli chcesz, możesz skorzystać tu również z wyników kolejnego zadania.

# Zadanie 3. (8+1p)

W tym zadaniu powinieneś sprawdzić, czy augmentacja danych może poprawić wyniki klasyfikacji, w której BERT jest traktowany jako ekstraktor cech. Mamy 3 osobno punktowane procedury generowania nowych wariantów recenzji:

**a)** Augmentacja mechaniczna (czyli wprowadzasz jakieś zniekształcenia w tekście, mogą to być na przykład literówki, zmiana wielkości liter, błędy związane z polskimi literami, etc). **(2p)**

**b)** Augmentacja modelem generatywnym, na przykład Papugą. Powinieneś generować recenzje, które bazują na oryginalnej recenzji, zachowując jej polarność (czyli to, czy jest pozytywna, czy negatywna). Zwróć uwagę, że „fantazja" modelu językowego nie musi tu być wadą – tak naprawdę to niekoniecznie w tej procedurze muszą powstawać poprawne teksty. **(3p)**

**c)** Ta procedura augmentacji powinna bazować na Word2Vec i zachowywać w miarę możliwości znaczenie tekstu. Należy wybrane słowa zamieniać na słowa bliskoznaczne, w tej samej formie gramatycznej (będzie to dokładniej omówione na kolejnym wykładzie). **(3p)**

Przykładowo recenzja:
- *Hotel ogólnie bardzo ładny.* mogłaby być zmieniona na *Pensjonat szczególnie bardzo piękny.*
- *Polecam wszystkim tego fizjoterapeutę!* na *Rekomenduję wszystkim tego ortopedę!*

Konieczne informacje gramatyczne pojawią się na wykładzie 8 (czyli najbliższym).

---

Każda recenzja powinna posłużyć do wygenerowania K innych recenzji (dobór K to Twoje zadanie), stąd należy generator napisać w ten sposób, by recenzje były tworzone niedeterministycznie.

Dla wybranych (lub wszystkich) procedur przeprowadź uczenie na zaugmentowanych danych za pomocą regresji logistycznej. Dodatkowo można uzyskać **1p premii**, jeżeli któraś z procedur da korzyść w porównaniu do oryginalnych danych (tzn. dzięki augmentacji uda się uzyskać lepszy wynik dla danych testowych).

W zadaniu do maksimum wlicza się **6p**.